In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import h5py

In [2]:
TRAIN_PATH = 'D:/UETCodeCamp/dataset/dataset/Images/Train'
VALIDATE_PATH = 'D:/UETCodeCamp/dataset/dataset/Images/Validate'
TEST_PATH = 'D:/UETCodeCamp/dataset/dataset/Images/Test'

In [4]:
import os

In [5]:
PATH = 'Models/InceptionResNetV2'

BASE_MODEL_BEST = os.path.join(PATH, 'base_model_best.hdf5')
BASE_MODEL_TRAINED = os.path.join(PATH, 'base_model_trained.hdf5')
BASE_MODEL_FIG = os.path.join(PATH, 'base_model_fig.jpg')

FINE_TUNE_MODEL_BEST = os.path.join(PATH, 'fine_tune_model_best.hdf5')
FINE_TUNE_MODEL_TRAINED = os.path.join(PATH, 'fine_tune_model_trained.hdf5')
FINE_TUNE_MODE_FIG = os.path.join(PATH, 'fine_tune_model_fig.jpg')

In [6]:
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 128

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
validate_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

In [8]:
generated_train_data = train_generator.flow_from_directory(TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_validate_data = validate_generator.flow_from_directory(VALIDATE_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE)

Found 18751 images belonging to 38 classes.
Found 2757 images belonging to 38 classes.
Found 5169 images belonging to 38 classes.


In [9]:
CLASSES = 38
INITIAL_EPOCHS = 15
FINE_TUNE_EPOCHS = 15
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
FINE_TUNE_AT = 711

In [10]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [11]:
pretrained_model = InceptionResNetV2(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)



219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
base_checkpointer = ModelCheckpoint(
    filepath=BASE_MODEL_BEST.replace('.hdf5', '.keras'), 
    save_best_only=True, 
    verbose=1
)

fine_tune_checkpointer = ModelCheckpoint(
    filepath=FINE_TUNE_MODEL_BEST.replace('.hdf5', '.keras'), 
    save_best_only=True,
    verbose=1, 
)


# Stop if no improvement after 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [14]:
for layer in pretrained_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    generated_train_data,
    validation_data = generated_validate_data,
    validation_steps = generated_validate_data.n // BATCH_SIZE,
    steps_per_epoch = generated_train_data.n // BATCH_SIZE,
    callbacks = [base_checkpointer, early_stopping],
    epochs = INITIAL_EPOCHS,
    verbose = 1,
)
model.save(BASE_MODEL_TRAINED)

Epoch 1/15


c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.2461 - loss: 2.8202 
Epoch 1: val_loss improved from inf to 1.71868, saving model to Models/InceptionResNetV2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 2992s 20s/step - accuracy: 0.2468 - loss: 2.8169 - val_accuracy: 0.4862 - val_loss: 1.7187
Epoch 2/15
  1/146 ━━━━━━━━━━━━━━━━━━━━ 44:31 18s/step - accuracy: 0.4219 - loss: 1.8363

c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 1.71868
146/146 ━━━━━━━━━━━━━━━━━━━━ 48s 205ms/step - accuracy: 0.4219 - loss: 1.8363 - val_accuracy: 0.4783 - val_loss: 1.7608
Epoch 3/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.4718 - loss: 1.7854 
Epoch 3: val_loss improved from 1.71868 to 1.51496, saving model to Models/InceptionResNetV2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 2633s 18s/step - accuracy: 0.4719 - loss: 1.7849 - val_accuracy: 0.5435 - val_loss: 1.5150
Epoch 4/15
  1/146 ━━━━━━━━━━━━━━━━━━━━ 25:15 10s/step - accuracy: 0.4766 - loss: 1.7179
Epoch 4: val_loss improved from 1.51496 to 1.46623, saving model to Models/InceptionResNetV2\base_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.4766 - loss: 1.7179 - val_accuracy: 0.5362 - val_loss: 1.4662
Epoch 5/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.5287 - loss: 1.5501 
Epoch 5: val_loss improved from 1.46623 to 1.45987, saving model to Models/InceptionResNetV2\base_mo

In [15]:
for layer in pretrained_model.layers[:FINE_TUNE_AT]: layer.trainable = False
for layer in pretrained_model.layers[FINE_TUNE_AT:]: layer.trainable = True
from tensorflow.keras.optimizers import SGD
model.compile(
    optimizer = SGD(learning_rate=1e-4, momentum=0.9), 
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)
history_fine = model.fit(
    generated_train_data,
    validation_data = generated_validate_data,
    validation_steps = generated_validate_data.n // BATCH_SIZE,
    steps_per_epoch = generated_train_data.n // BATCH_SIZE,
    epochs = TOTAL_EPOCHS,
    initial_epoch = history.epoch[-1],
    callbacks = [fine_tune_checkpointer, early_stopping],
    verbose = 1,
)
model.save(FINE_TUNE_MODEL_TRAINED)

Epoch 11/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.4288 - loss: 2.1140 
Epoch 11: val_loss improved from inf to 1.24048, saving model to Models/InceptionResNetV2\fine_tune_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 3314s 23s/step - accuracy: 0.4293 - loss: 2.1117 - val_accuracy: 0.6369 - val_loss: 1.2405
Epoch 12/30
  1/146 ━━━━━━━━━━━━━━━━━━━━ 50:05 21s/step - accuracy: 0.5547 - loss: 1.5444
Epoch 12: val_loss improved from 1.24048 to 1.22485, saving model to Models/InceptionResNetV2\fine_tune_model_best.keras
146/146 ━━━━━━━━━━━━━━━━━━━━ 34s 89ms/step - accuracy: 0.5547 - loss: 1.5444 - val_accuracy: 0.6522 - val_loss: 1.2248
Epoch 13/30
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.5980 - loss: 1.3921 
Epoch 13: val_loss did not improve from 1.22485
146/146 ━━━━━━━━━━━━━━━━━━━━ 2205s 15s/step - accuracy: 0.5981 - loss: 1.3919 - val_accuracy: 0.6403 - val_loss: 1.2357
Epoch 13: early stopping


In [16]:
from sklearn.metrics import classification_report
import numpy as np


# Sử dụng generator để dự đoán nhãn cho dữ liệu kiểm tra
generated_test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, shuffle=False)

# Dự đoán nhãn
predictions = model.predict(generated_test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = generated_test_data.classes

# Tính toán và in ra các chỉ số
class_labels = list(generated_test_data.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)


Found 5169 images belonging to 38 classes.


c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 458s 11s/step
                  precision    recall  f1-score   support

        Banh beo       0.83      0.71      0.77       129
    Banh bot loc       0.65      0.49      0.56       144
        Banh can       0.67      0.70      0.69       149
       Banh canh       0.42      0.40      0.41       193
      Banh chung       0.73      0.73      0.73       102
       Banh cuon       0.64      0.65      0.65       228
        Banh duc       0.48      0.21      0.29       133
        Banh gio       0.66      0.88      0.76       129
       Banh khot       0.81      0.66      0.73       167
         Banh mi       0.89      0.93      0.91       268
        Banh pia       0.84      0.87      0.85        89
        Banh tet       0.81      0.84      0.82       138
Banh trang nuong       0.77      0.81      0.79       159
        Banh xeo       0.75      0.81      0.78       235
      Bun bo Hue       0.51      0.70      0.59       306
 Bun dau mam tom       0.74   

c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\VuongQuan14\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa